In [1]:
import selenium
import pandas as pd
import re
import time

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.common.by import By

In [2]:
# Open Chrome
driver = webdriver.Chrome('C:/Users/li016/chromedriver/chromedriver.exe')

In [3]:
# input keyword
keyword = "데이터 분석"

# "원티드" 접속 by keyword
url = f"https://www.wanted.co.kr/search?query={keyword}"
driver.get(url)
driver.implicitly_wait(5)

# 신입 카테고리 설정 
# click "경력"
driver.find_element_by_css_selector("#__next > div.Search_className__7YET5 > div:nth-child(3) > div > div > div > div > div.FilterInWdlist_SelectedButtonGroup__3KRGs > button:nth-child(5)").click()
# select "신입"
select = Select(driver.find_element_by_css_selector("#MODAL_BODY > div > div > select"))
select.select_by_index(1)

# click "확인"
driver.find_element_by_css_selector("#__next > div.Search_className__7YET5 > div:nth-child(3) > div > div > div.Modal_root__1aywk > div.Modal_modalContent__X5ATV.YearsPopup_container__1ngcC > div > div.CommonFooter_container__qo8Wv > button").click()

# 포지션 수 get
position = driver.find_element_by_css_selector("#__next > div.Search_className__7YET5 > div:nth-child(3) > h2").text
pos_num = re.findall("[1-9]+",position)[0]
pos_num = int(pos_num)

In [4]:
info_list = []
# 범위 설정
for i in range(1, pos_num + 1):
    # 직무명 & 회사명 get
    job_info = driver.find_element_by_css_selector(
      f'#__next > div.Search_className__7YET5 > div:nth-child(3) > div > div > ul > li:nth-child({i}) > div > a > div'
    ).text
    job_info = job_info.split('\n')
    job_name = job_info[0]
    company_name = job_info[1]
    
    # 특정 채용 공고로 이동
    driver.find_element_by_css_selector(f'#__next > div.Search_className__7YET5 > div:nth-child(3) > div > div > ul > li:nth-child({i}) > div > a').click()
    driver.implicitly_wait(5)
    
    # 채용 공고 text 전체 get
    table_text = driver.find_element_by_class_name('JobDescription_JobDescription__3vTNT').text
    table_text = table_text.replace("\n", '').replace('[주요업무]', '')

    # 주요업무, 자격요건, 우대사항 get
    main_task = table_text.split("주요업무")[1].split("자격요건")[0]
    qualification = table_text.split("주요업무")[1].split("자격요건")[1].split("우대사항")[0]
    prefer_treat = table_text.split("주요업무")[1].split("자격요건")[1].split("우대사항")[1].split('혜택')[0]

    info_list.append([company_name, job_name, main_task, qualification, prefer_treat])
    
    driver.back()

In [5]:
result_df = pd.DataFrame(data=info_list, columns=['회사명', '직무명', '주요업무', '자격요건', '우대사항'])
result_df.head()

,회사명,직무명,주요업무,자격요건,우대사항
0,디셈버앤컴퍼니(핀트),핀트(Fint) 데이터 분석 담당자 (전문연구요원 가능),"[범주]• OLAP 분석, 대용량 로그 데이터 정제 및 분석• 수학/통계 또는 AI...",* 전문연구요원 신규 편입 / 전직 가능• 수학적/통계적 지식과 높은 수준의 논리적...,"• Tableau, Google Data Studio 등 BI 툴 활용 경험이 있으..."
1,더화이트커뮤니케이션,"자연어처리/AI개발, 데이터 분석 및 웹 Python개발(신입가능)",- 자체 구현한 NLP(자연어처리) 기술을 기반으로 추천 및 데이터처리 부분을 개발...,- Python 기반 REST-API 서버 개발 경험자이며 공고의 경력 또는 그에 ...,- NLP 관련 사항 개발 경험이 있으신 분- Docker 기반 환경의 서비스 구축...
2,아티언스,데이터 분석 컨설턴트(신입),ㆍ삼성 웹사이트 분석 지원ㆍ웹 로그 및 웹 트래픽 분석 지원ㆍ정기/비정기 분석 리포...,"ㆍ애널리틱스 툴(Adobe Analytics, Google Analytics) 이해...",ㆍ마케팅/통계학 전공자ㆍAdobe Analytics 혹은 Google Analyti...
3,데브시스터즈(Devsisters),[쿠키런: 오븐브레이크] 데이터 분석가,- 데이터웨어하우스 구축: 로그를 가공하여 분석에 필요한 데이터웨어하우스를 설계 및...,- Game Enthusiastic: 게임을 좋아하시는 분 (모바일 게임 뿐만 아니...,"- 3년 이상의 데이터 분석 실무 경력 (업계 무관)- Pandas, Numpy, ..."
4,지우컴퍼니(Lab543),데이터 분석가(신입가능),데이터 분석 및 리포트 작성• Python을 통한 데이터 수집• 데이터 품질 검수•...,"• 신입,경력(5년 이상)• Phython, R 등 하나 이상의 분석 언어 가능자•...","• kaggle 대회 유경험자• GA, GTM, Data Studio 활용 가능자•..."


In [ ]:
# 어떻게 하면 내가 편하게 볼 수 있을까?